In [1]:
#%cd /content/drive/MyDrive/nopeoplemachine
import drone.crop as dc
import drone2.nmsiou as nms
import drone.renorm640 as dr640
import drone.renorm1088 as dr1088
import cv2
import glob
import pandas as pd
import drone2.nmsiou as dn
import os
from tqdm import tqdm,trange
from time import sleep
import shutil


In [23]:
# crop image
# - file: image path
# - crop_path: path to save cropped images


original_img_path = r'train_offical'
crop_img_path = r'crop'
crop_label_path = r'crop'
def crop_image(file,crop_path,size):
    if size == 640:
        x1 = [0,370,910,1280]
        y1 = [0,440]

        x2 = [0,352,704]
        y2 = [0,80]

    elif size == 1088:
        x1 = [0,416,832]
        y1 = [0]

        x2 = [0,128,256]
        y2 = [0]

    # get image name
    img_name = file.split('\\')[-1]

    # read image
    img = cv2.imread(file)
    
    # get image width and height
    height, width, _ = img.shape

    # crop image
   
    if height == 1080 and width == 1920:
        for i in range(len(x1)):
            for j in range(len(y1)):
                crop_img = img[y1[j]:y1[j]+size, x1[i]:x1[i]+size]
                cv2.imwrite(crop_path +"\\img640\\"+ '{}_{}{}'.format(i+1,j+1,img_name), crop_img)
    elif height == 720 and width == 1344:
        for i in range(len(x2)):
            for j in range(len(y2)):
                crop_img = img[y2[j]:y2[j]+size, x2[i]:x2[i]+size]
                cv2.imwrite(crop_path +"\\img640\\"+ '{}_{}{}'.format(i+1,j+1,img_name), crop_img)
    else:
        return print('{} Image size not supported, image size = {},{}'.format(img_name,height,width))   
     


# read label data
# - file: original label path
# - sp: split character (default = ',')
def read_label(file,sp = ','):
    id = []
    x = []
    y = []
    w = []
    h = []
    conf = []
    
    with open(file,'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip('\n')
            s = line.split(sp)
            id.append(int(s[0]))
            x.append(int(s[1]))
            y.append(int(s[2]))
            w.append(int(s[3]))
            h.append(int(s[4]))
            # conf.append(float(s[5]))
    
    return id,x,y,w,h

# get image size
# - file: original image path
def image_size(file,img_type='.png'):
    image_path = file.split('.')[0] + img_type
    img = cv2.imread(image_path)
    height, width, channels = img.shape
    return height, width

# crop label
# - file: original label path
def crop_label(file,crop_label_path,size,img_type='.png'):
    # file name
    name = file.split('\\')[-1].split('.')[0]
    file_path = file.split(name)[0]
    file_path = file_path + name + ".txt"
    # crop path
    path = crop_label_path
    # read label data
    id,x,y,w,h = read_label(file_path)
    
    # get image size
    height, width = image_size(file,img_type)

    if size == 640:
        x1 = [370,910,1280]
        y1 = [440]

        x2 = [352,704]
        y2 = [80]

    elif size == 1088:
        x1 = [416,832]
        y1 = [0]

        x2 = [128,256]
        y2 = [0]
    if size == 640:
    # open file and write label data
        crop1 = open(path + '//img640//1_1' + name + '.txt', 'w+')
        crop2 = open(path + '//img640//2_1' + name + '.txt', 'w+')
        crop3 = open(path + '//img640//3_1' + name + '.txt', 'w+')
        crop4 = open(path + '//img640//4_1' + name + '.txt', 'w+')

        crop5 = open(path + '//img640//1_2' + name + '.txt', 'w+')
        crop6 = open(path + '//img640//2_2' + name + '.txt', 'w+')
        crop7 = open(path + '//img640//3_2' + name + '.txt', 'w+')
        crop8 = open(path + '//img640//4_2' + name + '.txt', 'w+')

        # crop label
        if height == 1080 and  width == 1920:
            for i in range(len(id)):
                if y[i] < 640:
                    if x[i] < 640:
                        crop1.write(str(id[i]) + ' ' + str(x[i]) + ' ' + str(y[i]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')
                    if 370 <= x[i] < 1010:
                        crop2.write(str(id[i]) + ' ' + str(x[i]-x1[0]) + ' ' + str(y[i]) + ' ' + str(w[i]) + ' ' + str(h[i]) + '\n')
                    if 910 <= x[i] < 1550:
                        crop3.write(str(id[i]) + ' ' + str(x[i]-x1[1]) + ' ' + str(y[i]) + ' ' + str(w[i]) + ' ' + str(h[i]) + '\n')
                    if size == 640:
                        if 1280 <= x[i] < 1920:
                            crop4.write(str(id[i]) + ' ' + str(x[i]-x1[2]) + ' ' + str(y[i]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')
                        
                if y[i] >= 440:
                    if x[i] < 640:
                        crop5.write(str(id[i]) + ' ' + str(x[i]) + ' ' + str(y[i]-y1[0]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')
                    if 370 <= x[i] < 1010:
                        crop6.write(str(id[i]) + ' ' + str(x[i]-x1[0]) + ' ' + str(y[i]-y1[0]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')
                    if 910 <= x[i] < 1550:
                        crop7.write(str(id[i]) + ' ' + str(x[i]-x1[1]) + ' ' + str(y[i]-y1[0]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')
                    if size == 640:
                        if 1280 <= x[i] < 1920:
                            crop8.write(str(id[i]) + ' ' + str(x[i]-x1[2]) + ' ' + str(y[i]-y1[0]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')

        elif height == 720 and width == 1344:
            for i in range(len(id)):
                if y[i] < 640:
                    if x[i] < 640:
                        crop1.write(str(id[i]) + ' ' + str(x[i]) + ' ' + str(y[i]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')
                    if 352 <= x[i] < 992:
                        crop2.write(str(id[i]) + ' ' + str(x[i]-x2[0]) + ' ' + str(y[i]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')
                    if 704 <= x[i] < 1344:
                        crop3.write(str(id[i]) + ' ' + str(x[i]-x2[1]) + ' ' + str(y[i]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')
                    
                    
                if y[i] >= 80:
                    if x[i] < 640:
                        crop5.write(str(id[i]) + ' ' + str(x[i]) + ' ' + str(y[i]-y2[0]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')
                    if 352 <= x[i] < 992:
                        crop6.write(str(id[i]) + ' ' + str(x[i]-x2[0]) + ' ' + str(y[i]-y2[0]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')
                    if 704 <= x[i] < 1344:
                        crop7.write(str(id[i]) + ' ' + str(x[i]-x2[1]) + ' ' + str(y[i]-y2[0]) + ' ' + str(w[i]) + ' ' + str(h[i])  + '\n')
        
        crop8.close()
        crop7.close()
        crop6.close()
        crop5.close()
        crop4.close()
        crop3.close()
        crop2.close()
        crop1.close()

    
    return

# remove empty label file with corresponding image
# - crop_label: cropped label
# - crop_image: cropped image
def remove_empty(crop_label,crop_image,img_type = '.png'):
    if os.stat(crop_label).st_size == 0:
        os.remove(crop_label)
        os.remove(crop_label.split('.')[0] + img_type)
    return

# Delete box outside of image
# - cropped_label: cropped label
def delete_outside_box(cropped_label):
    with open(cropped_label, 'r') as f:
        lines = f.readlines()
        f.close()
    with open(cropped_label, 'w') as f:
        for line in lines:
            id, x, y, w, h = line.split(' ')
            if int(x) + int(w) > 640 or int(y) + int(h) > 640:
                continue
            else:
                f.write(line)
        f.close()
    return

# Normalize label
# - cropped_label: cropped label
def normalize_label(cropped_label,img_type = '.png'): 
    with open(cropped_label, 'r') as f:
        lines = f.readlines()
        f.close()
    with open(cropped_label, 'w') as f:
        for line in lines:
            id, x, y, w, h = line.split()
            x = float(x) + float(w)/2
            y = float(y) + float(h)/2
            w = float(w)
            h = float(h)
            image = cropped_label.split('.')[0] + img_type
            height, width = image_size(image)
            x = x/width
            y = y/height
            w = w/width
            h = h/height
            f.write(str(id) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) + '\n')
        f.close()
    return 


# main
# - original_img_path: path to images
# - crop_img_path: path to save cropped images
# - crop_label_path: path to save cropped labels
def maincrop(img_type = '.png'):
    # crop image

    files = glob.glob(original_img_path + '\\*' + img_type)
    for file in tqdm(files,desc='Cropping image'):
        crop_image(file,crop_img_path,size=640)
        sleep(0.001)
    

    # crop label
    for file in tqdm(files,desc='Cropping label'):
        crop_label(file,crop_label_path,size=640,img_type='.png')
        sleep(0.001)

    # remove empty label file with corresponding image
    cropped_label640 = glob.glob(crop_label_path + '\\img640\\*' + '.txt') 
    cropped_image640 = glob.glob(crop_img_path + '\\img640\\*' + img_type)
    # Delete box outside of image
    for i in trange(len(cropped_label640),desc='Deleting box outside of image'):
        delete_outside_box(cropped_label640[i])
    # Normalize label
    for i in trange(len(cropped_label640),desc='Normalizing label'):
        normalize_label(cropped_label640[i],img_type = '.png')

        
    # try:
    #     for i in trange(len(cropped_label640),desc='Removing empty label'):
    #         remove_empty(cropped_label640[i],cropped_image640[i])
    # except FileNotFoundError:
    #     print('File not found: ' + cropped_label640[i] + ' or ' + cropped_image640[i])

maincrop()

Removing empty label:  75%|███████▌  | 6011/8000 [00:00<00:00, 15777.70it/s]

File not found: crop\img640\4_1img0012.txt or crop\img640\4_1img0014.png


<h2>normalize 1088

In [53]:
train_offical = r"train_offical"
train_normalize = r"crop\img1088"
def normalize1088():
    IMG = glob.glob(train_offical+'\\*.png')
    # print(IMG)
    for item in IMG:
        x = []
        y = []
        w = []
        h = []
        id = []
        TXT = item.split(".")[0]
        img = TXT.split('\\')[-1]
        # print(TXT)
        image = cv2.imread(item)
        (imgheight,imgwidth,rgb) = image.shape
        with open(TXT+".txt") as f:
            for line in f.readlines():
                s = line.split(',')
                id.append(int(s[0]))
                x.append(int(s[1]))
                y.append(int(s[2]))
                w.append(int(s[3]))
                h.append(int(s[4]))
        
        f2=open(train_normalize+'\\'+img+".txt",'w+')
        for a in range(len(id)):
            x_min = x[a]
            x_max = (x[a]+w[a])
            y_min = y[a]
            y_max = (y[a]+h[a])
            x_center = (x_min+x_max)/2
            y_center = (y_min+y_max)/2
            f2.write(str(id[a])+" "+str(x_center/imgwidth)+" "+str(y_center/imgheight)+" "+str(w[a]/imgwidth)+" "+str(h[a]/imgheight)+'\n')
        f2.close
    print('finish')

def copy():
    png = glob.glob(r"train_offical/*.png")
    crop1088path = r"crop\img1088"
    for i in png:
        shutil.copy(i,crop1088path)
    
def nor_main():
    normalize1088()
    copy()
nor_main()

<h2>train 640

In [1]:
%cd yolov7
!python train.py --weights weights\yolov7x_training.pt --cfg ..\machine\yolov7x_drone.yaml --data ..\machine\drone.yaml --device 0 --hyp data\hyp.scratch.custom640.yaml  --batch-size 9 --epoch 243 --freeze 87 --name first640 

d:\nopeoplemachine\yolov7
wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=9, bbox_interval=-1, bucket='', cache_images=False, cfg='..\\machine\\yolov7x_drone.yaml', data='..\\machine\\drone.yaml', device='0', entity=None, epochs=1, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data\\hyp.scratch.custom640.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='first640', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs\\train\\first640', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=9, upload_dataset=False, v5_metric=False, weights='weights\\yolov7x_training.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.001, lrf=0.2, momentum=0.937, weight_dec


train: WARNING: Ignoring corrupted image and/or label ..\crop\img640\2_1img0914.png: duplicate labels
train: WARNING: Ignoring corrupted image and/or label ..\crop\img640\2_2img0914.png: duplicate labels
train: WARNING: Ignoring corrupted image and/or label ..\crop\img640\3_1img0914.png: duplicate labels
train: WARNING: Ignoring corrupted image and/or label ..\crop\img640\3_2img0914.png: duplicate labels

autoanchor: Analyzing anchors... anchors/target = 4.37, Best Possible Recall (BPR) = 0.9995
                 all        1159       18756       0.199       0.265       0.202      0.0544
                 car        1159        8355       0.321       0.917       0.675       0.187
                 hov        1159         592       0.113      0.0557      0.0332      0.0105
              people        1159        7841           0           0     0.00542    0.000968
               motor        1159        1968       0.362      0.0853      0.0929      0.0189
Optimizer stripped from runs\trai

In [2]:
!python train.py --weights runs\train\first640\weights\best.pt --cfg ..\machine\yolov7x_drone.yaml --data ..\machine\drone.yaml --device 0 --hyp data\hyp.scratch.custom640.yaml  --batch-size 16 --epoch 100 --freeze 87 --name second640

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)
freezing model.0.conv.weight
freezing model.0.bn.weight
freezing model.0.bn.bias
freezing model.1.conv.weight
freezing model.1.bn.weight
freezing model.1.bn.bias
freezing model.2.conv.weight
freezing model.2.bn.weight
freezing model.2.bn.bias
freezing model.3.conv.weight
freezing model.3.bn.weight
freezing model.3.bn.bias
freezing model.4.conv.weight
freezing model.4.bn.weight
freezing model.4.bn.bias
freezing model.5.conv.weight
freezing model.5.bn.weight
freezing model.5.bn.bias
freezing model.6.conv.weight
freezing model.6.bn.weight
freezing model.6.bn.bias
freezing model.7.conv.weight
freezing model.7.bn.weight
freezing model.7.bn.bias
freezing model.8.conv.weight
freezing model.8.bn.weight
freezing model.8.bn.bias
freezing model.9.conv.weight
freezing model.9.bn.weight
freezing model.9.bn.bias
freezing model.10.conv.weight
freezing model.10.bn.weight
freezing model.10.bn.bias
freezing model.11

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=16, bbox_interval=-1, bucket='', cache_images=False, cfg='..\\machine\\yolov7x_drone.yaml', data='..\\machine\\drone.yaml', device='0', entity=None, epochs=1, evolve=False, exist_ok=False, freeze=[87], global_rank=-1, hyp='data\\hyp.scratch.custom640.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='second640', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs\\train\\second640', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=16, upload_dataset=False, v5_metric=False, weights='runs\\train\\first640\\weights\\best.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.001, lrf=0.2, momentum=0

<h2>train 1088(make new label)

In [5]:
!python train.py --weights weights\yolov7x_training.pt --cfg ..\machine\yolov7x_nopeoplemachine.yaml --data ..\machine\coco_nopeoplemachine.yaml --device 0  --batch-size 4 --epoch 100 --freeze 58 --name first1088

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)
freezing model.0.conv.weight
freezing model.0.bn.weight
freezing model.0.bn.bias
freezing model.1.conv.weight
freezing model.1.bn.weight
freezing model.1.bn.bias
freezing model.2.conv.weight
freezing model.2.bn.weight
freezing model.2.bn.bias
freezing model.3.conv.weight
freezing model.3.bn.weight
freezing model.3.bn.bias
freezing model.4.conv.weight
freezing model.4.bn.weight
freezing model.4.bn.bias
freezing model.5.conv.weight
freezing model.5.bn.weight
freezing model.5.bn.bias
freezing model.6.conv.weight
freezing model.6.bn.weight
freezing model.6.bn.bias
freezing model.7.conv.weight
freezing model.7.bn.weight
freezing model.7.bn.bias
freezing model.8.conv.weight
freezing model.8.bn.weight
freezing model.8.bn.bias
freezing model.9.conv.weight
freezing model.9.bn.weight
freezing model.9.bn.bias
freezing model.10.conv.weight
freezing model.10.bn.weight
freezing model.10.bn.bias
freezing model.11

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=4, bbox_interval=-1, bucket='', cache_images=False, cfg='..\\machine\\yolov7x_nopeoplemachine.yaml', data='..\\machine\\coco_nopeoplemachine.yaml', device='0', entity=None, epochs=1, evolve=False, exist_ok=False, freeze=[58], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='first1088', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs\\train\\first10882', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=4, upload_dataset=False, v5_metric=False, weights='weights\\yolov7x_training.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, moment

In [7]:
!python train.py --weights runs\train\first1088\weights\best.pt --cfg ..\machine\yolov7x_nopeoplemachine.yaml --data ..\machine\coco_nopeoplemachine.yaml --device 0  --batch-size 4 --epoch 100 --freeze 53 --name second1088

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=4, bbox_interval=-1, bucket='', cache_images=False, cfg='..\\machine\\yolov7x_nopeoplemachine.yaml', data='..\\machine\\coco_nopeoplemachine.yaml', device='0', entity=None, epochs=100, evolve=False, exist_ok=False, freeze=[53], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='second1088', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs\\train\\second1088', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=4, upload_dataset=False, v5_metric=False, weights='runs\\train\\first1088\\weights\\best.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, 

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)
freezing model.0.conv.weight
freezing model.0.bn.weight
freezing model.0.bn.bias
freezing model.1.conv.weight
freezing model.1.bn.weight
freezing model.1.bn.bias
freezing model.2.conv.weight
freezing model.2.bn.weight
freezing model.2.bn.bias
freezing model.3.conv.weight
freezing model.3.bn.weight
freezing model.3.bn.bias
freezing model.4.conv.weight
freezing model.4.bn.weight
freezing model.4.bn.bias
freezing model.5.conv.weight
freezing model.5.bn.weight
freezing model.5.bn.bias
freezing model.6.conv.weight
freezing model.6.bn.weight
freezing model.6.bn.bias
freezing model.7.conv.weight
freezing model.7.bn.weight
freezing model.7.bn.bias
freezing model.8.conv.weight
freezing model.8.bn.weight
freezing model.8.bn.bias
freezing model.9.conv.weight
freezing model.9.bn.weight
freezing model.9.bn.bias
freezing model.10.conv.weight
freezing model.10.bn.weight
freezing model.10.bn.bias
freezing model.11

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=4, bbox_interval=-1, bucket='', cache_images=False, cfg='..\\machine\\yolov7x_nopeoplemachine.yaml', data='..\\machine\\coco_nopeoplemachine.yaml', device='0', entity=None, epochs=1, evolve=False, exist_ok=False, freeze=[53], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='second1088', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs\\train\\second10882', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=4, upload_dataset=False, v5_metric=False, weights='runs\\train\\first1088\\weights\\best.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, l

In [2]:
!python train.py --weights runs\train\second1088\weights\best.pt --cfg ..\machine\yolov7x_nopeoplemachine.yaml --data ..\machine\coco_nopeoplemachine.yaml --device 0  --batch-size 4 --epoch 100 --freeze 48 --name third1088

d:\nopeoplemachine\yolov7
wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)
freezing model.0.conv.weight
freezing model.0.bn.weight
freezing model.0.bn.bias
freezing model.1.conv.weight
freezing model.1.bn.weight
freezing model.1.bn.bias
freezing model.2.conv.weight
freezing model.2.bn.weight
freezing model.2.bn.bias
freezing model.3.conv.weight
freezing model.3.bn.weight
freezing model.3.bn.bias
freezing model.4.conv.weight
freezing model.4.bn.weight
freezing model.4.bn.bias
freezing model.5.conv.weight
freezing model.5.bn.weight
freezing model.5.bn.bias
freezing model.6.conv.weight
freezing model.6.bn.weight
freezing model.6.bn.bias
freezing model.7.conv.weight
freezing model.7.bn.weight
freezing model.7.bn.bias
freezing model.8.conv.weight
freezing model.8.bn.weight
freezing model.8.bn.bias
freezing model.9.conv.weight
freezing model.9.bn.weight
freezing model.9.bn.bias
freezing model.10.conv.weight
freezing model.10.bn.weight
freezing model.10

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=4, bbox_interval=-1, bucket='', cache_images=False, cfg='..\\machine\\yolov7x_nopeoplemachine.yaml', data='..\\machine\\coco_nopeoplemachine.yaml', device='0', entity=None, epochs=1, evolve=False, exist_ok=False, freeze=[48], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='third1088', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs\\train\\third1088', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=4, upload_dataset=False, v5_metric=False, weights='runs\\train\\second1088\\weights\\best.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf

<h2>detect train1088

In [20]:
!python detect.py --weights runs\train\third1088\weights\best.pt --source ..\train_offical --path ..\new_label_train --img-size 1088 --save-conf --conf-thres [0.5,0.4,0.35,0.35]

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=[0.4, 0.4, 0.4, 0.4], device='', exist_ok=False, img_size=1088, iou_thres=0.5, name='new_label1088', no_trace=True, nosave=False, path='..\\new_label_train', project='runs/detect', save_conf=True, save_txt=True, source='..\\train_offical', update=False, view_img=False, weights=['runs\\train\\third1088\\weights\\best.pt'])
Fusing layers... 
IDetect.fuse
21 cars, Done. (22.9ms) Inference, (15.0ms) NMS
 The image with the result is saved in: ..\new_label_train//img0001.png
13 cars, Done. (23.9ms) Inference, (6.0ms) NMS
 The image with the result is saved in: ..\new_label_train//img0002.png
11 cars, Done. (23.0ms) Inference, (12.4ms) NMS
 The image with the result is saved in: ..\new_label_train//img0003.png
16 cars, Done. (22.9ms) Inference, (6.0ms) NMS
 The image with the result is saved in: ..\new_label_train//img0004.png
19 cars, Done. (23.9ms) Inference, (9.9ms) NMS
 The image with the result is saved in: ..\new_lab

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

c:\ProgramData\Anaconda3\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 362 layers, 70802658 parameters, 0 gradients, 188.0 GFLOPS


In [27]:
new = glob.glob(r'..\new_label_train\*.txt')
for item in new:
    with open(item, 'r+') as f:
        lines = f.readlines()
        f.close()
    with open(item, 'w') as f:
        for line in lines:
            id, x, y, w, h,conf = line.split()
            f.write(str(id) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) + '\n')
        f.close()
    shutil.move(item, r'..\crop\new_img1088')

Error: Destination path '..\crop\new_img1088\img0003.txt' already exists

<h2>use new label train 1088

In [29]:
!python train.py --weights weights\yolov7x_training.pt --cfg ..\machine\yolov7x_nopeoplemachine.yaml --data ..\machine\coco_nopeoplemachine_new.yaml --device 0 --batch-size 4 --epoch 100 --freeze 58 --name new_first1088 

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)

autoanchor: Analyzing anchors... anchors/target = 2.27, Best Possible Recall (BPR) = 0.9740. Attempting to improve anchors, please wait...
autoanchor: WARNING: Extremely small objects found. 440 of 35808 labels are < 3 pixels in size.
autoanchor: Running kmeans for 9 anchors on 35808 points...
autoanchor: thr=0.25: 1.0000 best possible recall, 6.66 anchors past thr
autoanchor: n=9, img_size=640, metric_all=0.417/0.803-mean/best, past_thr=0.504-mean: 4,7,  8,7,  9,13,  15,8,  12,20,  22,13,  21,31,  40,20,  59,43
autoanchor: thr=0.25: 1.0000 best possible recall, 6.99 anchors past thr
autoanchor: n=9, img_size=640, metric_all=0.432/0.812-mean/best, past_thr=0.507-mean: 4,7,  7,6,  8,11,  13,8,  10,18,  19,11,  18,22,  37,18,  50,38
autoanchor: New anchors saved to model. Update model *.yaml to use these anchors in the future.

                 all         100        4800       0.835      0.0909    

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=4, bbox_interval=-1, bucket='', cache_images=False, cfg='..\\machine\\yolov7x_nopeoplemachine.yaml', data='..\\machine\\coco_nopeoplemachine_new.yaml', device='0', entity=None, epochs=1, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='new_first1088', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs\\train\\new_first10884', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=4, upload_dataset=False, v5_metric=False, weights='weights\\yolov7x_training.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=

In [3]:
!python train.py --weights weights\yolov7x_training.pt --cfg ..\machine\yolov7x_nopeoplemachine.yaml --data ..\machine\coco_nopeoplemachine_new.yaml --device 0 --batch-size 4 --epoch 100 --freeze 53 --name new_second1088

d:\nopeoplemachine\yolov7
wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)
freezing model.0.conv.weight
freezing model.0.bn.weight
freezing model.0.bn.bias
freezing model.1.conv.weight
freezing model.1.bn.weight
freezing model.1.bn.bias
freezing model.2.conv.weight
freezing model.2.bn.weight
freezing model.2.bn.bias
freezing model.3.conv.weight
freezing model.3.bn.weight
freezing model.3.bn.bias
freezing model.4.conv.weight
freezing model.4.bn.weight
freezing model.4.bn.bias
freezing model.5.conv.weight
freezing model.5.bn.weight
freezing model.5.bn.bias
freezing model.6.conv.weight
freezing model.6.bn.weight
freezing model.6.bn.bias
freezing model.7.conv.weight
freezing model.7.bn.weight
freezing model.7.bn.bias
freezing model.8.conv.weight
freezing model.8.bn.weight
freezing model.8.bn.bias
freezing model.9.conv.weight
freezing model.9.bn.weight
freezing model.9.bn.bias
freezing model.10.conv.weight
freezing model.10.bn.weight
freezing model.10

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=4, bbox_interval=-1, bucket='', cache_images=False, cfg='..\\machine\\yolov7x_nopeoplemachine.yaml', data='..\\machine\\coco_nopeoplemachine_new.yaml', device='0', entity=None, epochs=1, evolve=False, exist_ok=False, freeze=[53], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='new_second1088', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs\\train\\new_second1088', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=4, upload_dataset=False, v5_metric=False, weights='weights\\yolov7x_training.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lr

In [4]:
!python train.py --weights weights\yolov7x_training.pt --cfg ..\machine\yolov7x_nopeoplemachine.yaml --data ..\machine\coco_nopeoplemachine_new.yaml --device 0 --batch-size 4 --epoch 100 --freeze 48 --name new_third1088

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)
freezing model.0.conv.weight
freezing model.0.bn.weight
freezing model.0.bn.bias
freezing model.1.conv.weight
freezing model.1.bn.weight
freezing model.1.bn.bias
freezing model.2.conv.weight
freezing model.2.bn.weight
freezing model.2.bn.bias
freezing model.3.conv.weight
freezing model.3.bn.weight
freezing model.3.bn.bias
freezing model.4.conv.weight
freezing model.4.bn.weight
freezing model.4.bn.bias
freezing model.5.conv.weight
freezing model.5.bn.weight
freezing model.5.bn.bias
freezing model.6.conv.weight
freezing model.6.bn.weight
freezing model.6.bn.bias
freezing model.7.conv.weight
freezing model.7.bn.weight
freezing model.7.bn.bias
freezing model.8.conv.weight
freezing model.8.bn.weight
freezing model.8.bn.bias
freezing model.9.conv.weight
freezing model.9.bn.weight
freezing model.9.bn.bias
freezing model.10.conv.weight
freezing model.10.bn.weight
freezing model.10.bn.bias
freezing model.11

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

Namespace(adam=False, artifact_alias='latest', batch_size=4, bbox_interval=-1, bucket='', cache_images=False, cfg='..\\machine\\yolov7x_nopeoplemachine.yaml', data='..\\machine\\coco_nopeoplemachine_new.yaml', device='0', entity=None, epochs=1, evolve=False, exist_ok=False, freeze=[48], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='new_third1088', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs\\train\\new_third1088', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=4, upload_dataset=False, v5_metric=False, weights='weights\\yolov7x_training.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=

<h2>crop public

In [16]:

def public_crop_image(file,crop_path,size):
    if size == 640:
        x1 = [0,370,910,1280]
        y1 = [0,440]

        x2 = [0,352,704]
        y2 = [0,80]

    elif size == 1088:
        x1 = [0,416,832]
        y1 = [0]

        x2 = [0,128,256]
        y2 = [0]

    # get image name
    img_name = file.split('\\')[-1]

    # read image
    img = cv2.imread(file)
    
    # get image width and height
    height, width, _ = img.shape

    # crop image
    if size == 640:
        if height == 1080 and width == 1920:
            for i in range(len(x1)):
                for j in range(len(y1)):
                    crop_img = img[y1[j]:y1[j]+size, x1[i]:x1[i]+size]
                    cv2.imwrite(crop_path +"\\img640\\"+ '{}_{}{}'.format(i+1,j+1,img_name), crop_img)
        elif height == 720 and width == 1344:
            for i in range(len(x2)):
                for j in range(len(y2)):
                    crop_img = img[y2[j]:y2[j]+size, x2[i]:x2[i]+size]
                    cv2.imwrite(crop_path +"\\img640\\"+ '{}_{}{}'.format(i+1,j+1,img_name), crop_img)
        else:
            return print('{} Image size not supported, image size = {},{}'.format(img_name,height,width))   
    else:
        if height == 1080 and width == 1920:
            for i in range(len(x1)):
                for j in range(len(y1)):
                    crop_img = img[y1[j]:y1[j]+size, x1[i]:x1[i]+size]
                    cv2.imwrite(crop_path +"\\img1088\\"+ '{}_{}{}'.format(i+1,j+1,img_name), crop_img)
        elif height == 720 and width == 1344:
            for i in range(len(x2)):
                for j in range(len(y2)):
                    crop_img = img[y2[j]:y2[j]+size, x2[i]:x2[i]+size]
                    cv2.imwrite(crop_path +"\\img1088\\"+ '{}_{}{}'.format(i+1,j+1,img_name), crop_img)
        else:
            return print('{} Image size not supported, image size = {},{}'.format(img_name,height,width))

public_img_path = r'public'
crop_path = r'public_crop'
files = glob.glob(public_img_path + '\\*.png')
for file in tqdm(files,desc='Cropping image'):
    public_crop_image(file,crop_path,size=640)
    public_crop_image(file,crop_path,size=1088)


Cropping image: 100%|██████████| 1000/1000 [07:56<00:00,  2.10it/s]


<h2>detect 640

In [6]:
!python detect.py --weights runs\train\second640\weights\best.pt --source ..\public_crop\img640 --path ..\output_label_path\img640 --img-size 640 --save-conf --iou 0.5 

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=[0.4, 0.4, 0.4, 0.4], device='', exist_ok=False, img_size=640, iou_thres=0.5, name='exp', no_trace=True, nosave=False, path='..\\output_label_path\\img640', project='runs/detect', save_conf=True, save_txt=True, source='..\\public_crop\\img640', update=False, view_img=False, weights=['runs\\train\\second640\\weights\\best.pt'])
Fusing layers... 
IDetect.fuse
6 cars, Done. (18.9ms) Inference, (5.0ms) NMS
 The image with the result is saved in: ..\output_label_path\img640//1_1img1001.png
Done. (18.9ms) Inference, (0.0ms) NMS
 The image with the result is saved in: ..\output_label_path\img640//1_1img1002.png
Done. (18.0ms) Inference, (1.0ms) NMS
 The image with the result is saved in: ..\output_label_path\img640//1_1img1003.png
17 cars, Done. (19.9ms) Inference, (18.0ms) NMS
 The image with the result is saved in: ..\output_label_path\img640//1_1img1004.png
1 car, Done. (18.9ms) Inference, (2.0ms) NMS
 The image with the

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

c:\ProgramData\Anaconda3\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 362 layers, 70802658 parameters, 0 gradients, 188.0 GFLOPS


<h2>detect 1088

In [9]:
!python detect.py --weights runs\train\new_third1088\weights\best.pt --source ..\public_crop\img1088 --path ..\output_label_path\img1088 --img-size 1088 --save-conf --conf-thres [0.5,0.4,0.35,0.35]

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres='[0.5,0.4,0.35,0.35]', device='', exist_ok=False, img_size=1088, iou_thres=0.5, name='exp', no_trace=False, nosave=False, path='..\\output_label_path\\img1088', project='runs/detect', save_conf=True, save_txt=True, source='..\\public_crop\\img1088', update=False, view_img=False, weights=['runs\\train\\new_third1088\\weights\\best.pt'])
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

16 cars, Done. (39.9ms) Inference, (17.0ms) NMS
 The image with the result is saved in: ..\output_label_path\img1088//1_1img1001.png
28 cars, Done. (36.9ms) Inference, (15.0ms) NMS
 The image with the result is saved in: ..\output_label_path\img1088//1_1img1002.png
3 cars, Done. (35.9ms) Inference, (4.0ms) NMS
 The image with the result is saved in: ..\output_label_path\img1088//1_1img1003.png
3 cars, Done. (36.9ms) Inference, (5.0ms) NMS
 The image with the result is sav

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

c:\ProgramData\Anaconda3\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 362 layers, 70802658 parameters, 0 gradients, 188.0 GFLOPS


Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres='[0.5,0.4,0.35,0.35]', device='', exist_ok=False, img_size=1088, iou_thres=0.5, name='exp', no_trace=False, nosave=False, path='..\\output_label_path\\img1088', project='runs/detect', save_conf=True, save_txt=True, source='..\\public_crop\\img1088', update=False, view_img=False, weights=['runs\\train\\new_third1088\\weights\\best.pt'])
Fusing layers... 
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

16 cars, Done. (46.9ms) Inference, (32.9ms) NMS
 The image with the result is saved in: ..\output_label_path\img1088//1_1img1001.png
28 cars, Done. (49.9ms) Inference, (17.0ms) NMS
 The image with the result is saved in: ..\output_label_path\img1088//1_1img1002.png
3 cars, Done. (33.9ms) Inference, (4.0ms) NMS
 The image with the result is saved in: ..\output_label_path\img1088//1_1img1003.png
3 cars, Done. (53.8ms) Inference, (6.0ms) NMS
 The image with the result is sav

YOLOR  2022-10-9 torch 1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11263.6875MB)

c:\ProgramData\Anaconda3\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 362 layers, 70802658 parameters, 0 gradients, 188.0 GFLOPS


<h2>renorm and nms 640

In [4]:
# renormalize label data
# - file: output label path after detection
# - csv_path: path to save csv file
%cd ../
output_label_path = r'output_label_path\img640'
origin_img_path = r'public'
csv_path = r'csv\img640'
no_nms = r'no_nms\img640'

def renorm(output_label,csv_path,sp = ','):
    name = output_label.split("\\")[-1].split(".")[0]
    id = []
    x1 = []
    y1 = []
    w1 = []
    h1 = []
    conf = []
    with open(output_label) as f:
        for line in f.readlines():
            s = line.split(' ')
            id.append(int(s[0]))
            x1.append(float(s[1]))
            y1.append(float(s[2]))
            w1.append(float(s[3]))
            h1.append(float(s[4]))
            conf.append(float(s[5]))
    
    f2=open(csv_path +'\\'+ name + ".csv",'w+') #crop的已經renomalize的label
    for a in range(len(id)):
        w = int(w1[a] * 640)
        h = int(h1[a] * 640)
        x = int((x1[a] * 640)-(w/2))
        y = int((y1[a] * 640)-(h/2))

        f2.write(str(id[a])+","+str(x)+","+str(y)+","+str(w)+","+str(h)+","+str(conf[a])+'\n')
    f2.close()
    

# read label data
# - file: output label path after detection
def read_label(file,sp = ','):
    id = []
    x = []
    y = []
    w = []
    h = []
    conf = []
    with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip('\n')
            s = line.split(sp)
            id.append(int(s[0]))
            x.append(int(s[1]))
            y.append(int(s[2]))
            w.append(int(s[3]))
            h.append(int(s[4]))
            conf.append(float(s[5]))
    return id,x,y,w,h,conf

# get image size
# - file: original image path
def image_size(file,img_type='.png'):
    image_path = file.split('.')[0] + img_type
    img = cv2.imread(image_path)
    height, width, channels = img.shape
    return height, width
           

# crop label
# - csv_file: label path after renormalization
# - original_img_path: original image path
def re_label(csv_file,origin_img_path,img_type='.png'):
    # file name
    name = csv_file.split('\\')[-1].split('.')[0]
    label = name[:3]

    # file path
    path = csv_file.split(name)[0]
    # read label data
    id,x,y,w,h,conf = read_label(csv_file)
    # get image size
    try:
        height, width = image_size(origin_img_path +'//'+ name[3:] + img_type)
    except:
        print(origin_img_path +'//'+ name[3:] + img_type)

    # open file and write
    crop1 = open(path +  name + '.csv', 'w+')
    crop2 = open(path +  name + '.csv', 'w+')
    crop3 = open(path +  name + '.csv', 'w+')
    crop4 = open(path +  name + '.csv', 'w+')

    crop5 = open(path + name + '.csv', 'w+')
    crop6 = open(path + name + '.csv', 'w+')
    crop7 = open(path + name + '.csv', 'w+')
    crop8 = open(path + name + '.csv', 'w+')

    # crop label and delete label which is out of range
    try:
        if height == 1080 and width == 1920:
            for i in range(len(id)):
                if label in ['1_1','2_1','3_1','4_1']:
                    if y[i] < 560:
                        if label == '1_1':
                            if x[i] < 560 or x[i] + w[i]< 560:
                                crop1.write(str(id[i]) + ',' + str(x[i]) + ',' + str(y[i]) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                        elif label == '2_1':
                            if (80 <= x[i] < 560) or (80 <= (x[i] + w[i]) < 560):
                                crop2.write(str(id[i]) + ',' + str(x[i]+370) + ',' + str(y[i]) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                        elif label == '3_1':
                            if (80 <= x[i] < 560) or (80 <= (x[i] + w[i]) < 560):
                                crop3.write(str(id[i]) + ',' + str(x[i]+910) + ',' + str(y[i]) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                        elif label == '4_1':
                            if (80 <= x[i]) or (80 <= (x[i] + w[i])):
                                crop4.write(str(id[i]) + ',' + str(x[i]+1280) + ',' + str(y[i]) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                elif label in ['1_2','2_2','3_2','4_2']:        
                    if y[i] >= 80:
                        if label == '1_2':
                            if (x[i] < 560) or ((x[i] + w[i])< 560):
                                crop5.write(str(id[i]) + ',' + str(x[i]) + ',' + str(y[i]+440) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                        elif label == '2_2':
                            if (80 <= x[i] < 560) or (80 <= (x[i] + w[i]) < 560):
                                crop6.write(str(id[i]) + ',' + str(x[i]+370) + ',' + str(y[i]+440) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                        elif label == '3_2':
                            if (80 <= x[i] < 560) or (80 <= (x[i] + w[i]) < 560):
                                crop7.write(str(id[i]) + ',' + str(x[i]+910) + ',' + str(y[i]+440) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                        elif label == '4_2':
                            if (80 <= x[i]) or (80 <= (x[i] + w[i])):
                                crop8.write(str(id[i]) + ',' + str(x[i]+1280) + ',' + str(y[i]+440) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')

        elif height == 720 and width == 1344:
            for i in range(len(id)):
                if label in ['1_1','2_1','3_1','4_1']:
                    if y[i] < 590:
                        if label == '1_1':
                            if (x[i] < 560) or ((x[i] + w[i])< 560):
                                crop1.write(str(id[i]) + ',' + str(x[i]) + ',' + str(y[i]) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                        elif label == '2_1':
                            if (80 <= x[i] < 560) or (80 <= (x[i] + w[i]) < 560):
                                crop2.write(str(id[i]) + ',' + str(x[i]+352) + ',' + str(y[i]) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                        elif label == '3_1':
                            if (80 <= x[i]) or (80 <= (x[i] + w[i])):
                                crop3.write(str(id[i]) + ',' + str(x[i]+704) + ',' + str(y[i]) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                        
                elif label in ['1_2','2_2','3_2','4_2']: 
                    if y[i] >= 50:
                        if label == '1_2':
                            if (x[i] < 560) or ((x[i] + w[i])< 560):
                                crop5.write(str(id[i]) + ',' + str(x[i]) + ',' + str(y[i]+80) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                        elif label == '2_2':
                            if (80 <= x[i] < 560) or (80 <= (x[i] + w[i]) < 560):
                                crop6.write(str(id[i]) + ',' + str(x[i]+352) + ',' + str(y[i]+80) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
                        elif label == '3_2':
                            if (80 <= x[i]) or (80 <= (x[i] + w[i])):
                                crop7.write(str(id[i]) + ',' + str(x[i]+704) + ',' + str(y[i]+80) + ',' + str(w[i]) + ',' + str(h[i]) + ',' + str(conf[i]) + '\n')
        
        crop8.close()
        crop7.close()
        crop6.close()
        crop5.close()
        crop4.close()
        crop3.close()
        crop2.close()
        crop1.close()
    except:
        print(origin_img_path +'//'+ name[3:] + img_type)
    return

# combine labels
# - csv_label: path to csv label
# - final: path to final output
def combine_labels(csv_label,no_nms):
    img_name = csv_label.split("\\")[-1].split(".")[0]
    # img_name = img_name.split(".")[0]
    name = img_name[3:]
    label = img_name[:3]
    # write new file with same name
    f2=open(no_nms +'\\'+ name + ".csv",'a+')
    with open(csv_label) as f:
        for line in f.readlines():
            new = line.strip('\n')
            f2.write(new + ',' + label + '\n')
    f2.close()
    return

def nms():
    dn.put_1500csv(modelpath = no_nms,mergepath = r"nms\model640")
# main
# - output_label_path: path to output label
# - origin_img_path: path to origin image
# - csv_path: path to csv file after renormalize
# - no_nms: path to no_nms output
def main640():
    label_path = glob.glob(output_label_path + '\\*.txt')
    # renormalize labels
    for file in tqdm(label_path,desc='Renormalize detection labels'):
        renorm(file,csv_path)
        sleep(0.001)

    csv_label = glob.glob(csv_path + '\\*.csv')
    # Transform labels to original
    for file in tqdm(csv_label,desc='Transform detection labels'):
        re_label(file,origin_img_path)
        sleep(0.001)

    # combine labels
    for file in tqdm(csv_label,desc='Combine detection labels'):
        combine_labels(file,no_nms)
        sleep(0.001)
    nms()
main640()


Transform detection labels:  24%|██▎       | 1224/5164 [01:11<03:17, 19.90it/s]

public//img0001.png
public//img0001.png
public//img0002.png
public//img0002.png
public//img0003.png
public//img0003.png


Combine detection labels: 100%|██████████| 5164/5164 [01:21<00:00, 63.02it/s]
d:\nopeoplemachine\drone2\nmsiou.py:11: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2985.)
  box2 = box2.T


['no_nms\\img640\\img0001.csv', 'no_nms\\img640\\img0002.csv', 'no_nms\\img640\\img0003.csv', 'no_nms\\img640\\img1001.csv', 'no_nms\\img640\\img1002.csv', 'no_nms\\img640\\img1003.csv', 'no_nms\\img640\\img1004.csv', 'no_nms\\img640\\img1005.csv', 'no_nms\\img640\\img1006.csv', 'no_nms\\img640\\img1007.csv', 'no_nms\\img640\\img1008.csv', 'no_nms\\img640\\img1009.csv', 'no_nms\\img640\\img1010.csv', 'no_nms\\img640\\img1011.csv', 'no_nms\\img640\\img1012.csv', 'no_nms\\img640\\img1013.csv', 'no_nms\\img640\\img1014.csv', 'no_nms\\img640\\img1015.csv', 'no_nms\\img640\\img1016.csv', 'no_nms\\img640\\img1017.csv', 'no_nms\\img640\\img1019.csv', 'no_nms\\img640\\img1020.csv', 'no_nms\\img640\\img1021.csv', 'no_nms\\img640\\img1022.csv', 'no_nms\\img640\\img1023.csv', 'no_nms\\img640\\img1024.csv', 'no_nms\\img640\\img1025.csv', 'no_nms\\img640\\img1026.csv', 'no_nms\\img640\\img1027.csv', 'no_nms\\img640\\img1028.csv', 'no_nms\\img640\\img1029.csv', 'no_nms\\img640\\img1030.csv', 'no_nms

<h2>renorm and nms 1088

In [2]:
csv_path = r'csv\img1088'
no_nms = r'no_nms\img1088'
original_crop_img = r'public_crop\img1088'
detect_img = r'output_label_path\img1088'
def inward():
    filelistpath = csv_path+"\\" + "*.csv"
    filelist = glob.glob(filelistpath)
    count =0
    for item in filelist:
        dellist = []
        file = pd.read_csv(item,header = 0,names = ["id","x","y","w","h","conf","IDcode"])
        Fname = item.split("\\")[-1]
        cn = Fname.split(".")[0]
        image = cv2.imread("public"+ "\\"+ cn + ".png")
        (imgheight,imgwidth,rgb) = image.shape
        for i in range(len(file["id"])):
            if imgwidth == 1920:
                if (file["IDcode"][i] == "3_1") and ((int(file["x"][i]) ) < 960):
                    dellist.append(i)
                elif (file["IDcode"][i] == "2_1") and ((int(file["x"][i]) + int(file["w"][i]))> 1376) :
                    dellist.append(i)
                elif (file["IDcode"][i] == "2_1") and  (int(file["x"][i]) < 544) :
                    dellist.append(i)
                elif (file["IDcode"][i] == "1_1") and  ((int(file["x"][i]) + int(file["w"][i]))> 960) :
                    dellist.append(i)
            else:
                if (file["IDcode"][i] == "3_1") and (int(file["x"][i]) < 384):
                    dellist.append(i)
                elif (file["IDcode"][i] == "2_1") and ((int(file["x"][i])+ int(file["w"][i])) > 1088) :
                    dellist.append(i)
                elif (file["IDcode"][i] == "2_1") and  (int(file["x"][i]) < 256) :
                    dellist.append(i)
                elif (file["IDcode"][i] == "1_1") and  ((int(file["x"][i]) + int(file["w"][i])) > 960) :
                    dellist.append(i)
        count += len(file["id"])
        file.drop(file.index[dellist],axis = 0,inplace = True)
        file.to_csv(no_nms + "\\" + cn + ".csv" ,index = 0)        

def nms():
    dn.put_1500csv(modelpath = no_nms,mergepath = r"nms\model1088")

def main1088():
    path_IMG = glob.glob(original_crop_img + "\\*.png")
    for item in path_IMG:
        img = item.split("\\")[-1]
        img = img.split(".")[0]
        num = img[:3]
        name = img[3:]
        id = []
        x1 = []
        y1 = []
        w1 = []
        h1 = []
        conf = []
        image = cv2.imread(item)
        (imgheight,imgwidth,rgb) = image.shape
        try:
            with open(detect_img + "\\" + img + ".txt" ) as f:
                for line in f.readlines():
                    s = line.split(' ')
                    id.append(int(s[0]))
                    x1.append(float(s[1]))
                    y1.append(float(s[2]))
                    w1.append(float(s[3]))
                    h1.append(float(s[4]))
                    conf.append(float(s[5][:-1]))
            f2=open(csv_path+ "\\" + name + ".csv",'a+')
            for a in range(len(id)):
                w = int(w1[a] * imgwidth)
                h = int(h1[a] * imgheight)
                x = int((x1[a]*imgwidth)-(w/2))
                y = int((y1[a]*imgheight)-(h/2))
                if num =='3_1':
                    if imgheight == 1080 and imgwidth == 1088:
                        x = x + 832
                    else:
                        x = x + 256
                if num == '2_1':
                    if imgheight == 1080 and imgwidth == 1088:
                        x = x + 416
                    else:
                        x = x + 128
                
                f2.write(str(id[a])+","+str(x)+","+str(y)+","+str(w)+","+str(h)+","+str(conf[a])+','+str(num)+'\n')
            f2.close()
        except:
            continue
    inward()
    nms()
main1088()

d:\nopeoplemachine\drone2\nmsiou.py:11: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2985.)
  box2 = box2.T


img1001
[1, 3, 6, 8, 10, 13, 15, 18, 20, 22, 27, 30, 34, 42, 45, 49, 53, 54, 55, 64, 65, 66, 67, 76, 79, 81, 84, 86, 88, 91, 93, 96, 98, 100]
---------------------------------------------------------------------------------
img1002
[0, 4, 7, 9, 13, 14, 15, 17, 19, 21, 22, 23, 25, 29, 32, 34, 38, 39, 40, 42, 44, 46, 47, 48, 49, 51, 66, 68, 83, 87, 88, 92, 94, 98, 101, 103, 107, 108, 109, 111, 113, 115, 116, 117, 119, 123, 126, 128, 132, 133, 134, 136, 138, 140, 141, 142]
---------------------------------------------------------------------------------
img1003
[1, 3, 4, 5, 7, 8, 13, 15]
---------------------------------------------------------------------------------
img1004
[0, 3, 16, 19]
---------------------------------------------------------------------------------
img1005
[0, 1]
---------------------------------------------------------------------------------
img1006
[3, 4, 5, 6]
---------------------------------------------------------------------------------
img1008
[0, 1, 2]
---

KeyboardInterrupt: 

<h2> nms 640,1088

In [10]:
import pandas as pd
import math
import torch

mergepath = r"nms\merge"
resultpath = r"result.csv"

def bbox_iou(box1, box2, x1y1x2y2=True, GIoU=False, DIoU=False, CIoU=False, eps=1e-7):
    # Returns the IoU of box1 to box2. box1 is 4, box2 is nx4
    box2 = box2.T
    
    # Get the coordinates of bounding boxes
    if x1y1x2y2:  # x1, y1, x2, y2 = box1
        b1_x1, b1_y1, b1_x2, b1_y2 = box1[0], box1[1], box1[2], box1[3]
        b2_x1, b2_y1, b2_x2, b2_y2 = box2[0], box2[1], box2[2], box2[3]
    else:  # transform from xywh to xyxy
        b1_x1, b1_x2 = box1[0] - box1[2] / 2, box1[0] + box1[2] / 2
        b1_y1, b1_y2 = box1[1] - box1[3] / 2, box1[1] + box1[3] / 2
        b2_x1, b2_x2 = box2[0] - box2[2] / 2, box2[0] + box2[2] / 2
        b2_y1, b2_y2 = box2[1] - box2[3] / 2, box2[1] + box2[3] / 2

    # Intersection area
    inter = (torch.min(b1_x2, b2_x2) - torch.max(b1_x1, b2_x1)).clamp(0) * \
            (torch.min(b1_y2, b2_y2) - torch.max(b1_y1, b2_y1)).clamp(0)

    # Union Area
    w1, h1 = b1_x2 - b1_x1, b1_y2 - b1_y1 + eps
    w2, h2 = b2_x2 - b2_x1, b2_y2 - b2_y1 + eps
    union = w1 * h1 + w2 * h2 - inter + eps

    iou = inter / union

    if GIoU or DIoU or CIoU:
        cw = torch.max(b1_x2, b2_x2) - torch.min(b1_x1, b2_x1)  # convex (smallest enclosing box) width
        ch = torch.max(b1_y2, b2_y2) - torch.min(b1_y1, b2_y1)  # convex height
        if CIoU or DIoU:  # Distance or Complete IoU https://arxiv.org/abs/1911.08287v1
            c2 = cw ** 2 + ch ** 2 + eps  # convex diagonal squared
            rho2 = ((b2_x1 + b2_x2 - b1_x1 - b1_x2) ** 2 +
                    (b2_y1 + b2_y2 - b1_y1 - b1_y2) ** 2) / 4  # center distance squared
            if DIoU:
                return iou - rho2 / c2  # DIoU
            elif CIoU:  # https://github.com/Zzh-tju/DIoU-SSD-pytorch/blob/master/utils/box/box_utils.py#L47
                v = (4 / math.pi ** 2) * torch.pow(torch.atan(w2 / (h2 + eps)) - torch.atan(w1 / (h1 + eps)), 2)
                with torch.no_grad():
                    alpha = v / (v - iou + (1 + eps))
                return iou - (rho2 / c2 + v * alpha)  # CIoU
        else:  # GIoU https://arxiv.org/pdf/1902.09630.pdf
            c_area = cw * ch + eps  # convex area
            return iou - (c_area - union) / c_area  # GIoU
    else:
        return iou  # IoU
def countIOU(text,Fname,mergepath):
    del_index = []                                                                                                                   # correcting the index that need to remove
    same = []                                                                                                                   # correcting the distance that need to remove (it's doesn't used)
    for i in range(len(text)):
        for j in range(i,len(text)):
            if (i != j) and (text["IDcode"][i] != text["IDcode"][j]) and (text["id"][i] == text["id"][j]): 
                box1 =torch.tensor([text["x"][i],text["y"][i],text["x"][i]+text["w"][i],text["y"][i]+text["h"][i]])
                box2 =torch.tensor([text["x"][j],text["y"][j],text["x"][j]+text["w"][j],text["y"][j]+text["h"][j]])                 # Comparing the IDcode , id is the same or not and let i not equal to j
                # print(box1)
                # print("..........................")
                # print(box2)
                iou = bbox_iou(box1,box2)
                if iou >= 0.5:  
                    iou = iou.item()                                                                                                     # if IOU <= 0.5 ,del
                    if text["conf"][i] > text["conf"][j]:                                                                            # Comparing the confidance which is larger                 
                        del_index.append(j)
                    
                    elif text["conf"][j] > text["conf"][i]:
                        del_index.append(i)
                        
                    else:
                        iarea = text["w"][i]*text["h"][i]
                        jarea = text["w"][j]*text["h"][j]
                        area = min([iarea,jarea])
                        if area == iarea:
                            if iarea != jarea:
                                del_index.append(j)
                                
                            else:
                                if (i or j)  not in del_index:
                                 del_index.append(j)   
                        else:
                            if iarea != jarea:
                                del_index.append(i)
             
                            else:
                                if (i or j)  not in del_index:
                                 del_index.append(j) 

                    iarea = text["w"][i]*text["h"][i]
                    jarea = text["w"][j]*text["h"][j]
                    if iarea != jarea :
                        if text["conf"][i] >= text["conf"][j]:
                            same.append(j)
                        else:
                            same.append(i)

    same = sorted(list(set(same)))                    
    del_index = sorted(list(set(del_index)))
    text.drop(text.index[del_index],axis = 0,inplace = True)
    text.to_csv(mergepath + "\\" + Fname ,index = 0)                                                                             # del the index
    return del_index,same                                                                                                                 # return index which has been del            

def merge():
    f2 = open(resultpath,'w+') # result csv
    files = glob.glob(mergepath+"\*.csv")    # new 1500 csv files
    for file in files:
        name = file.split("\\")[-1]
        name = name.split(".")[0]                           # for instance : name = img1001
        df = pd.read_csv(file,header = 0)
        for i in range(len(df["id"])):
            f2.write(name + "," + str(df["id"][i]) + "," + str(df["x"][i]) + "," + str(df["y"][i]) + "," + str(df["w"][i]) + "," + str(df["h"][i]) + "\n")

    f2.close()

def final_nms():
    Cpath = r"nms\model640" # first file path
    Fpath = r"nms\model1088"  # Second file path
    Flist = glob.glob(Fpath + "\\*.csv")  
    Clist = glob.glob(Cpath + "\\*.csv")         # all first file path csv
    CHlist = ["id","x","y","w","h","conf","IDcode"]
    FHlist = ["id","x","y","w","h","conf","IDcode"]
    count = 0
    tcount = 0
    for i in range(len(Clist)):
        Fpath = r"nms\model1088"
        Cname = Clist[i].split("\\")[-1]
        Fpath = Fpath + "\\" + Cname
        dellist = []
        try:
            Ftext = pd.read_csv(Fpath, header=0, names=FHlist)
            
            for j in range(len(Ftext["id"])):
                Ftext["IDcode"][j] = Ftext["IDcode"][j] + "F"

                if  (int(Ftext["id"][j]) == 2) or (int(Ftext["id"][j]) ==  3):
                    dellist.append(j)
            Ftext.drop(Ftext.index[dellist],axis = 0,inplace = True)
            
            Ctext = pd.read_csv(Clist[i], header=0, names=CHlist)
            text = pd.concat([Ftext,Ctext],axis=0)
        except:
            Ctext = pd.read_csv(Clist[i], header=0, names=CHlist)
            text = Ctext
            print("except: " + Cname)
        text = text.reset_index(drop=True)                                  # reset the index
        cf = Cname.split(".")[0]  
        d,s = countIOU(text,Cname,mergepath)   
        a = len(d)-len(s)   
        if a<0:
            break         
        count += a
        tcount += len(d)                           # for instance : name = img1001
        print(cf)
        print(d)
        #print(a)                                      # Use the function countIOU(text,Fname) or count(text,Fname) 
        print("---------------------------------------------------------------------------------")
    print("del total: " + str(tcount))
    print("del diffient: " + str(count))

final_nms()






<h2> 建立最後的結果

In [ ]:
merge()